ACLARACIÓN: PARA EJECUTAR CORRECTAMENTE EL CÓDIGO DE ESTE NOTEBOOK, ES NECESARIO CAMBIAR LAS RUTAS EN LA QUE SE GUARDAN Y CARGAN LOS DISTINTOS ARCHIVOS QUE SE USAN Y/O CREAN.

En las siguientes celdas se crean las 24 topologías disitntas para las redes de entrenamineto y las 6 topologías distintas para las redes de test, en ambos casos, siendo creadas mediante el modelo Barabasí-Albert (redes sintéticas)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

# Parámetros para las redes
num_nodos = [1000, 3000]  # Número de nodos
grados_medios = [2, 4, 8]  # Grado medio de cada nodo
num_semillas = 4  # Número de semillas aleatorias

# Almacenar las redes generadas
redes = []

for n in num_nodos:
    for k in grados_medios:
        for seed in range(num_semillas):
            # El número de enlaces que se añaden desde un nuevo nodo a nodos existentes
            m = k // 2  # Se divide por 2 porque en el modelo Barabasi-Albert cada enlace añade 2 al grado total
            # Establecer la semilla para la generación aleatoria
            random_seed = random.randint(0, 10000)  # Genera una semilla aleatoria
            # Generar la red Barabasi-Albert con la semilla especificada
            G = nx.barabasi_albert_graph(n, m, seed=random_seed)
            redes.append(G)
            # Opcional: Visualizar la red
            # nx.draw(G, with_labels=False, node_size=30)
            # plt.show()

print(f"Se han generado {len(redes)} redes Barabasi-Albert con semillas aleatorias.")


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Parámetros para las redes
num_nodos = [2000, 4000]  # Número de nodos
grados_medios = [2, 4, 8]  # Grado medio de cada nodo

# Almacenar las redes generadas
redes_test = []

for n in num_nodos:
    for k in grados_medios:
        # El número de enlaces que se añaden desde un nuevo nodo a nodos existentes
        m = k // 2  # Se divide por 2 porque en el modelo Barabasi-Albert cada enlace añade 2 al grado total
        # Generar la red Barabasi-Albert
        G = nx.barabasi_albert_graph(n, m)
        redes_test.append(G)
        # Opcional: Visualizar la red
        #nx.draw(G, with_labels=False, node_size=30)
        #plt.show()

print(f"Se han generado {len(redes_test)} redes Barabasi-Albert.")


Las siguientes celdas son auxiliares para ayudar a usar siempre las mismas redes de entrenamiento y test, dada la complejidad del entrenamiento de estos modelos.

import networkx as nx

def guardar_redes(redes, prefijo):
    for i, G in enumerate(redes):
        nx.write_adjlist(G, f"{prefijo}_red_{i}.adjlist")

guardar_redes(redes, "train")
#guardar_redes(redes_test, "test")


In [1]:
import networkx as nx
def cargar_redes(prefijo, num_redes):
    redes = []
    for i in range(num_redes):
        try:
            path = f"{prefijo}_red_{i}.adjlist"
            G = nx.read_adjlist(path)
            redes.append(G)
        except FileNotFoundError:
            print(f"No se encontró el archivo: {path}")
            break
    return redes

num_redes_train = 24 
num_redes_test = 6  

redes = cargar_redes("train", num_redes_train)
redes_test = cargar_redes("test", num_redes_test)

if not redes or not redes_test:
    print(f"hay algun problema.")


Las siguientes celdas crean las funciones correspondientes para simular la difusión de los modelos SIR e IC, que se usan para crear los ground truth para el entrenamiento de nuestros modelos.

In [2]:
import networkx as nx
import numpy as np
import random

def simulate_SIR_and_generate_labels(G, initial_infecteds, p=0.2, r=0.02, max_steps=100):
    # Inicializar todos los nodos a susceptibles
    status = {node: 'S' for node in G.nodes()}

    # Infectar los nodos iniciales
    for node in initial_infecteds:
        status[node] = 'I'

    for step in range(max_steps):
        new_status = dict(status)
        for node in G.nodes():
            if status[node] == 'I':
                # Intentar infectar a los vecinos
                for neighbor in G.neighbors(node):
                    if status[neighbor] == 'S' and random.random() < p:
                        new_status[neighbor] = 'I'
                # Recuperar el nodo infectado
                if random.random() < r:
                    new_status[node] = 'R'
        if status == new_status:
            break  # Termina si no hay cambios
        status = new_status

    # Generar etiquetas basadas en el estado final de los nodos
    labels = [1 if status[node] == 'I' else 0 for node in G.nodes()]
    return labels



In [3]:
def simulate_IC_and_generate_labels(G, initial_infecteds, p=0.2, max_steps=100):
    # Inicializar todos los nodos como susceptibles excepto los infectados iniciales
    status = {node: 'S' for node in G.nodes()}
    for node in initial_infecteds:
        status[node] = 'I'
    
    newly_infected = set(initial_infecteds)
    for step in range(max_steps):
        if not newly_infected:
            break  # Terminar si no hay nuevos infectados
        
        new_status = dict(status)
        current_newly_infected = set()
        for infected_node in newly_infected:
            for neighbor in G.neighbors(infected_node):
                if status[neighbor] == 'S' and random.random() < p:
                    new_status[neighbor] = 'I'
                    current_newly_infected.add(neighbor)
        
        status = new_status
        newly_infected = current_newly_infected
        
        if not newly_infected:
            break  # Termina si no hay nuevos infectados en este paso
    
    # Generar etiquetas basadas en el estado final de los nodos
    labels = [1 if status[node] == 'I' else 0 for node in G.nodes()]
    return labels


# Ahora vamos a generar las etiquetas con el modelo IC para todas las redes
# Establecer el tamaño de nodos infectados iniciales a 30
#initial_infected_size = 30
#results_by_network_IC = []

#for G in redes:
    # Seleccionar nodos infectados iniciales basado en el tamaño de 30
    #initial_infecteds = random.sample(list(G.nodes()), initial_infected_size)
    # Simular el modelo SIR
    #results = simulate_IC_and_generate_labels(G, initial_infecteds)
    # Guardar los resultados
    #results_by_network_IC.append(results)


import sys
sys.path.append('C:\\Users\\manue\\struc2vec')

import sys
sys.path.append('C:\\Users\\manue\\struc2vec\\src')

En las dos siguientes celdas se ejecuta el metodo de incrustacion de embeddings en las redes de entrenamiento, previamente creando la lista de aristas de las redes.

In [ ]:
from src.struc2vec import Graph
import networkx as nx

for i, G in enumerate(redes):
    nx.write_edgelist(G, f"red_sintetica_{i}.edgelist")

In [ ]:
import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'  # Asumiendo que main.py está en el directorio src

for i, _ in enumerate(redes):
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}.edgelist --output embeddings_{i}.emb --dimensions 64 --walk-length 70 --num-walks 8 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")


Las siguientes celdas se encargan de la creación de las redes reales a través de los ficheros con los datos de las matrices de adyacencia.

In [ ]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open('C:\\Users\\manue\\soc-wiki-Vote.mtx', 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [ ]:
import networkx as nx
from torch_geometric.utils import from_networkx

# Crear un grafo vacío en NetworkX
G = nx.Graph()

file_path = 'C:\\Users\\manue\\fb-pages-food.edges'

# Leer las aristas desde el archivo 'fb-pages-food.edges'
with open(file_path, 'r') as file:
    for line in file:
        if not line.startswith('%'):  # Ignorar líneas de comentarios
            parts = line.split(',')
            if len(parts) >= 2:
                source = int(parts[0])
                target = int(parts[1])
                G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [4]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

file_path = 'C:\\Users\\manue\\web-edu.mtx'

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open(file_path, 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [5]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

file_path = 'C:\\Users\\manue\\ia-fb-messages.mtx'

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open(file_path, 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [6]:
import torch
import networkx as nx
from torch_geometric.utils import from_networkx

# Nombre del archivo de texto

file_path = 'C:\\Users\\manue\\USairport500.txt'

# Crear un grafo en NetworkX
G = nx.Graph()

# Abrir el archivo y leer las líneas
with open(file_path, 'r') as file:
    for line in file:
        # Dividir la línea en sus componentes y convertirlos en enteros
        source, target, weight = map(int, line.split())
        # Agregar los nodos y las aristas al grafo
        G.add_edge(source, target, weight=weight)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [7]:
import networkx as nx
from torch_geometric.utils import from_networkx

# Ruta al archivo que contiene las aristas

file_path = 'C:\\Users\\manue\\soc-hamsterster.edges'

# Crear un grafo vacío
G = nx.Graph()

# Leer el archivo de aristas y agregarlas al grafo
with open(file_path, 'r') as file:
    for line in file:
        if not line.startswith('%'):  # Ignorar líneas de comentarios
            parts = line.split()
            if len(parts) >= 2:
                source = int(parts[0])
                target = int(parts[1])
                G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [8]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

# Crear un grafo vacío en NetworkX
G = nx.Graph()

file_path = 'C:\\Users\\manue\\soc-sign-bitcoinalpha.csv'

# Leer el archivo CSV y agregar las aristas al grafo
with open(file_path, 'r') as file:
    for line in file:
        parts = line.strip().split(',')
        source = int(parts[0])
        target = int(parts[1])
        G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [9]:
import networkx as nx
from scipy.io import mmread
import torch
from torch_geometric.utils import from_networkx

# Leer el archivo MTX
matrix = mmread('C:\\Users\\manue\\econ-mahindas.mtx')

# Convertir la matriz dispersa en un grafo NetworkX
G = nx.Graph(matrix)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


Las siguientes celdas, al igual que otras anteriores ya comentadas, ejecutan el método struc2vec en las redes de test.

In [ ]:
from src.struc2vec import Graph
import networkx as nx

for i, G in enumerate(redes_test):
    nx.write_edgelist(G, f"red_sintetica_{i}_test.edgelist")

In [ ]:
import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'  # Asumiendo que main.py está en el directorio src

# Ejecuta struc2vec solo para las primeras 6 redes
for i in range(6):
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}_test.edgelist --output embeddings_{i}_test.emb --dimensions 64 --walk-length 70 --num-walks 8 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")



In [ ]:
from src.struc2vec import Graph
import networkx as nx
import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'

def save_edgelist(graph, filename):
    with open(filename, 'w') as file:
        for edge in graph.edge_index.T.tolist():
            file.write(f"{edge[0]} {edge[1]}\n")

indice_inicio = len(redes_test) - 8

for i in range(indice_inicio, len(redes_test)):
    save_edgelist(redes_test[i], f"red_sintetica_{i}_test.edgelist")


for i in range(indice_inicio, len(redes_test)):    
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}_test.edgelist --output embeddings_{i}_test.emb --dimensions 128 --walk-length 80 --num-walks 10 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")


Las siguientes celdas simulan la difusión con las funciones anteriormente comentadas en las 24 topologías de red de entrenamiento para los distintos tamaños escogidos (10,30 y 50) para el conjunto de semillas iniciales.

In [11]:
import torch
from torch_geometric.data import Data
import numpy as np
import networkx as nx
import random

data_list_SIR = []  # Para almacenar objetos Data de todas las redes
seed_sizes = [10, 30, 50]  # Tamaños de semillas iniciales

for i in range(len(redes)):  # Asumimos que 'redes' contiene todas las redes
    # Cargar la red sintética
    G = nx.read_edgelist(f'red_sintetica_{i}.edgelist', nodetype=int)
    
    # Cargar los embeddings generados por struc2vec o algún otro método
    embeddings = np.loadtxt(f'embeddings_{i}.emb', skiprows=1)
    node_features = embeddings[:, 1:]  # Las características (embeddings) de los nodos
    
    # Crear un tensor de características
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Crear listas de aristas
    edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    
    for seed_size in seed_sizes:
        # Generar etiquetas usando la función de simulación SIR o IC modificada
        initial_infecteds = random.sample(list(G.nodes()), seed_size)  # Escoger nodos al azar como infectados iniciales
        labels = simulate_SIR_and_generate_labels(G, initial_infecteds)
        
        y = torch.tensor(labels, dtype=torch.long)
        
        # Crear el objeto Data con x, edge_index y y
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list_SIR.append(data)


In [12]:
import torch
from torch_geometric.data import Data
import numpy as np
import networkx as nx
import random

data_list_SI = []  # Para almacenar objetos Data de todas las redes
seed_sizes = [10, 30, 50]  # Tamaños de semillas inSIiales

for i in range(len(redes)):  # Asumimos que 'redes' contiene todas tus redes
    # Cargar la red sintética
    G = nx.read_edgelist(f'red_sintetica_{i}.edgelist', nodetype=int)
    
    # Cargar los embeddings generados por struc2vec o algún otro método
    embeddings = np.loadtxt(f'embeddings_{i}.emb', skiprows=1)
    node_features = embeddings[:, 1:]  # Las características (embeddings) de los nodos
    
    # Crear un tensor de características
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Crear listas de aristas
    edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    
    for seed_size in seed_sizes:
        # Generar etiquetas usando la función de simulación SI modificada
        initial_infecteds = random.sample(list(G.nodes()), seed_size)  # Escoger nodos al azar como infectados iniciales
        labels = simulate_SIR_and_generate_labels(G, initial_infecteds, r=0)
        
        y = torch.tensor(labels, dtype=torch.long)
        
        # Crear el objeto Data con x, edge_index y y
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list_SI.append(data)


In [13]:
import torch
from torch_geometric.data import Data
import numpy as np
import networkx as nx
import random

data_list_IC = []  # Para almacenar objetos Data de todas las redes
seed_sizes = [10, 30, 50]  # Tamaños de semillas iniciales

for i in range(len(redes)):  # Asumimos que 'redes' contiene todas tus redes
    # Cargar la red sintética
    G = nx.read_edgelist(f'red_sintetica_{i}.edgelist', nodetype=int)
    
    # Cargar los embeddings generados por struc2vec o algún otro método
    embeddings = np.loadtxt(f'embeddings_{i}.emb', skiprows=1)
    node_features = embeddings[:, 1:]  # Las características (embeddings) de los nodos
    
    # Crear un tensor de características
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Crear listas de aristas
    edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    
    for seed_size in seed_sizes:
        # Generar etiquetas usando la función de simulación IC o IC modificada
        initial_infecteds = random.sample(list(G.nodes()), seed_size)  # Escoger nodos al azar como infectados iniciales
        labels = simulate_IC_and_generate_labels(G, initial_infecteds)
        
        y = torch.tensor(labels, dtype=torch.long)
        
        # Crear el objeto Data con x, edge_index y y
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list_IC.append(data)


Las siguientes celdas se encargan de separar las redes de entrenamiento en 2 conjuntos, 70% para train y 30% para validación. Además se ejecutan comandos auxiliares para poder realizar lo anterior y para poder realizar el entrenamiento en 5 scripts distintos para ser utilizados en 5 GPUs distintas. En este código veríamos como se ejecuta el primer script, siendo el resto de scripts iguales, cambiando el valor de new_index a 1, 2, 3 o 4 respectivamente.

In [15]:
import os

sizes = [10, 30, 50]
num_redes = 24

# Asegúrate de que existe un directorio para los embeddings modificados
os.makedirs('modified_embeddings', exist_ok=True)

# Nuevo índice para el archivo de salida
new_index = 0

for i in range(num_redes):
    original_file = f'embeddings_{i}.emb'
    # Leer el contenido original si es necesario
    original_content = open(original_file, 'r').read()
    for size in sizes:
        new_file = f'modified_embeddings/embeddings_{new_index}.emb'
        with open(new_file, 'w') as file:
            file.write(original_content)
        new_index += 1  # Incrementa el índice para cada nuevo archivo


In [ ]:
import numpy as np
import os
import shutil
import pickle

# Supongamos que tienes 72 redes y sus índices de archivos van del 0 al 71
indices = np.arange(72)
np.random.shuffle(indices)

# Divide los índices en entrenamiento y validación
train_indices = indices[:int(0.7 * len(indices))]  # 70% para entrenamiento
val_indices = indices[int(0.7 * len(indices)):]  # 30% para validación

# Suponiendo que `data_list_SIR`, `data_list_SI`, y `data_list_IC` son listas de tus 72 redes
train_data_SIR = [data_list_SIR[i] for i in train_indices]
val_data_SIR = [data_list_SIR[i] for i in val_indices]

train_data_SI = [data_list_SI[i] for i in train_indices]
val_data_SI = [data_list_SI[i] for i in val_indices]

train_data_IC = [data_list_IC[i] for i in train_indices]
val_data_IC = [data_list_IC[i] for i in val_indices]



# Crea directorios para guardar los datos divididos y los chunks
os.makedirs('train_data_chunks', exist_ok=True)
os.makedirs('val_data', exist_ok=True)

# Guarda cada chunk en un archivo separado
chunk_size = 10  # Número de redes por chunk
train_data_chunks_SIR = [train_data_SIR[i:i + chunk_size] for i in range(0, len(train_data_SIR), chunk_size)]
train_data_chunks_SI = [train_data_SI[i:i + chunk_size] for i in range(0, len(train_data_SI), chunk_size)]
train_data_chunks_IC = [train_data_IC[i:i + chunk_size] for i in range(0, len(train_data_IC), chunk_size)]

index_to_chunk_map = {index: idx // chunk_size for idx, index in enumerate(train_indices)}

for i in range(len(train_data_chunks_SIR)):
    chunk_dir = f'train_data_chunks/chunk_{i}'
    os.makedirs(chunk_dir, exist_ok=True)
    
    with open(f'{chunk_dir}/train_data_chunk_SIR.pkl', 'wb') as f:
        pickle.dump(train_data_chunks_SIR[i], f)
        
    with open(f'{chunk_dir}/train_data_chunk_SI.pkl', 'wb') as f:
        pickle.dump(train_data_chunks_SI[i], f)  
    
    with open(f'{chunk_dir}/train_data_chunk_IC.pkl', 'wb') as f:
        pickle.dump(train_data_chunks_IC[i], f)            

# Copia los archivos de embeddings correspondientes a los directorios de entrenamiento y validación
for idx in train_indices:
    chunk_idx = index_to_chunk_map[idx]
    shutil.copy(f'modified_embeddings/embeddings_{idx}.emb', f'train_data_chunks/chunk_{chunk_idx}/')
    
for idx in val_indices:
    shutil.copy(f'modified_embeddings/embeddings_{idx}.emb', 'val_data/')

    
    


La siguiente celda se queda comentada pues es una celda auxiliar usada una vez el entrenamiento ya había sido realizado en las distinas GPUs.

import numpy as np

# Lista de archivos de embeddings en val_data
embedding_files = ['embeddings_0.emb', 'embeddings_10.emb', 'embeddings_11.emb', 'embeddings_14.emb', 'embeddings_15.emb', 
                   'embeddings_16.emb', 'embeddings_17.emb', 'embeddings_18.emb', 'embeddings_2.emb', 'embeddings_20.emb', 
                   'embeddings_21.emb', 'embeddings_22.emb', 'embeddings_23.emb', 'embeddings_24.emb', 'embeddings_27.emb', 
                   'embeddings_28.emb', 'embeddings_29.emb', 'embeddings_3.emb', 'embeddings_30.emb', 'embeddings_32.emb', 
                   'embeddings_33.emb', 'embeddings_35.emb', 'embeddings_36.emb', 'embeddings_37.emb', 'embeddings_38.emb', 
                   'embeddings_39.emb', 'embeddings_4.emb', 'embeddings_40.emb', 'embeddings_41.emb', 'embeddings_42.emb', 
                   'embeddings_43.emb', 'embeddings_44.emb', 'embeddings_45.emb', 'embeddings_46.emb', 'embeddings_47.emb', 
                   'embeddings_48.emb', 'embeddings_49.emb', 'embeddings_5.emb', 'embeddings_50.emb', 'embeddings_52.emb', 
                   'embeddings_53.emb', 'embeddings_54.emb', 'embeddings_57.emb', 'embeddings_58.emb', 'embeddings_59.emb', 
                   'embeddings_60.emb', 'embeddings_61.emb', 'embeddings_62.emb', 'embeddings_63.emb', 'embeddings_64.emb', 
                   'embeddings_65.emb', 'embeddings_66.emb', 'embeddings_67.emb', 'embeddings_68.emb', 'embeddings_69.emb', 
                   'embeddings_70.emb', 'embeddings_71.emb', 'embeddings_8.emb', 'embeddings_9.emb']

# Extraer los índices de los archivos de embeddings
val_indices = [int(f.split('_')[1].split('.')[0]) for f in embedding_files]

# Crear las listas de validación para SIR, SI e IC
val_data_SIR = [data_list_SIR[i] for i in val_indices]
val_data_SI = [data_list_SI[i] for i in val_indices]
val_data_IC = [data_list_IC[i] for i in val_indices]

print("val_data_SIR:", val_data_SIR)
print("val_data_SI:", val_data_SI)
print("val_data_IC:", val_data_IC)



In [ ]:
# Leer el archivo de datos de entrenamiento
chunk_index = 0  # Cambia este índice para los otros archivos
chunk_dir = f'train_data_chunks/chunk_{chunk_index}'

train_data_file_SIR = f'{chunk_dir}/train_data_chunk_SIR.pkl'
train_data_file_SI = f'{chunk_dir}/train_data_chunk_SI.pkl'
train_data_file_IC = f'{chunk_dir}/train_data_chunk_IC.pkl'

# Cargar los datos de entrenamiento desde los archivos
with open(train_data_file_SIR, 'rb') as f:
    train_data_SIR = pickle.load(f)    

with open(train_data_file_SI, 'rb') as f:
    train_data_SI = pickle.load(f) 

with open(train_data_file_IC, 'rb') as f:
    train_data_IC = pickle.load(f)

La siguiente celda crea la arquitectura de capas para el primer modelo de GNN a usar, GCN.

In [ ]:
import torch
from torch_geometric.nn import GCNConv, BatchNorm
from torch.nn import Dropout  # Corregido

class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = BatchNorm(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = BatchNorm(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = BatchNorm(hidden_dim)
        self.conv4 = GCNConv(hidden_dim, output_dim)
        self.dropout = Dropout(dropout_rate)  # Usar el parámetro de dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        return x



Las siguientes celdas ejecutan el entrenamiento de GCN en las redes de los distintos modelos de difusión (SIR, SI e IC)

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GCN_SIR = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SIR):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GNN(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_SIR_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GCN_SIR.append(model_path)
        print(f"Modelo SIR {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GCN_SI = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SI):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GNN(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_SI_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GCN_SI.append(model_path)
        print(f"Modelo SI {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GCN_IC = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_IC):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GNN(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_IC_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GCN_IC.append(model_path)
        print(f"Modelo IC {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



Las siguientes funciones calculan el total de nodos infectados a través de los modelos de difusión en la red que se le pasa como argumento.

In [17]:
import random

def simular_difusion_SIR(G, semillas, p=0.05, r=0.02, max_steps=100):
    status = {node: 'S' for node in G.nodes()}
    for node in semillas:
        status[node] = 'I'
    
    #print(f"Estado inicial (semillas infectadas): {semillas}")

    for step in range(max_steps):
        new_status = dict(status)
        infectados = 0
        recuperados = 0
        for node in G.nodes():
            if status[node] == 'I':
                for neighbor in G.neighbors(node):
                    if status[neighbor] == 'S' and random.random() < p:
                        new_status[neighbor] = 'I'
                        infectados += 1
                if random.random() < r:
                    new_status[node] = 'R'
                    recuperados += 1
        status = new_status
        #print(f"Paso {step + 1}: Infectados nuevos = {infectados}, Recuperados = {recuperados}")
        
        if infectados == 0 and recuperados == 0:
            #print("No hay más cambios en el estado de los nodos.")
            break
    
    total_infectados = sum(1 for state in status.values() if state in ['I', 'R'])
    #print(f"Total infectados al final: {total_infectados}")
    return total_infectados


In [18]:
import numpy as np

def simular_difusion_IC(G, semillas, p=0.05, max_steps=100):
    status = {node: 'S' for node in G.nodes()}
    for node in semillas:
        if node not in G:
            print(f"El nodo {node} no está en el grafo.")
        status[node] = 'I'
    
    newly_infected = set(semillas)
    #print(f"Estado inicial (semillas infectadas): {semillas}")

    for step in range(max_steps):
        if not newly_infected:
     #       print("No hay nuevos infectados.")
            break
        
        new_status = dict(status)
        current_newly_infected = set()
        for infected_node in newly_infected:
            if infected_node not in G:
                continue  # Salta a la siguiente iteración si el nodo no existe
            for neighbor in G.neighbors(infected_node):
                if status[neighbor] == 'S' and np.random.random() < p:
                    new_status[neighbor] = 'I'
                    current_newly_infected.add(neighbor)
        
        status = new_status
        newly_infected = current_newly_infected
        #print(f"Paso {step + 1}: Nuevos infectados = {len(newly_infected)}")
        
    total_infectados = sum(1 for state in status.values() if state == 'I')
    #print(f"Total infectados al final: {total_infectados}")
    return total_infectados


La siguiente es una celda auxiliar usada una vez se ha realizado el entrenamiento de los distintos modelos. Por tanto, se deja comentada.

# Inicialización de listas para cada combinación de modelo y tipo de difusión
modelos_paths_GCN_SIR = []
modelos_paths_GCN_SI = []
modelos_paths_GCN_IC = []

modelos_paths_GAT_SIR = []
modelos_paths_GAT_SI = []
modelos_paths_GAT_IC = []

modelos_paths_GraphSAGE_SIR = []
modelos_paths_GraphSAGE_SI = []
modelos_paths_GraphSAGE_IC = []

# Lista con los nombres de los archivos de modelos
model_names = [
    "Nuevo_model_GAT_IC_0_0_lr0.001_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_0_1_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GAT_IC_0_2_lr0.01_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_0_3_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_0_4_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_0_5_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_0_6_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_0_7_lr0.001_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_0_8_lr0.001_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_0_9_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_1_0_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_1_1_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_2_lr0.001_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_1_3_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_4_lr0.001_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_1_5_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_6_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_7_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_8_lr0.001_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_1_9_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_2_0_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_2_1_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_2_2_lr0.005_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_2_3_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_2_4_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_2_5_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_2_6_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_2_7_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_2_8_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GAT_IC_2_9_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_3_0_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_3_1_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_3_2_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_3_3_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_3_4_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_3_5_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_3_6_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_3_7_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_3_8_lr0.001_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_3_9_lr0.001_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GAT_IC_4_0_lr0.001_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_4_1_lr0.001_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_IC_4_2_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_4_3_lr0.01_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_4_4_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_IC_4_5_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_IC_4_6_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GAT_IC_4_7_lr0.01_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_GAT_IC_4_8_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_IC_4_9_lr0.001_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_0_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_1_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_2_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_4_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_5_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_6_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_7_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_8_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_0_9_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_1_0_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_1_1_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_1_2_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_1_3_lr0.005_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SI_1_4_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_1_5_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_1_6_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_1_7_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_1_8_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_1_9_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_0_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_1_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_2_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_3_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_4_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_5_lr0.001_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_2_6_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_7_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_2_8_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_2_9_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_3_0_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_3_1_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_3_2_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_3_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_3_4_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_3_5_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_3_6_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_3_7_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_3_8_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_3_9_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SI_4_0_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_1_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_2_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_4_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SI_4_5_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_6_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_7_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_8_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SI_4_9_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_0_0_lr0.001_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_0_1_lr0.001_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_0_2_lr0.001_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_0_3_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_0_4_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_0_5_lr0.001_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_0_6_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_0_7_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_0_8_lr0.005_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_0_9_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_1_0_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_1_1_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_1_2_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_1_3_lr0.001_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_1_4_lr0.001_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_1_5_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_1_6_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_1_7_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_1_8_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_1_9_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_2_0_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_2_1_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_2_2_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_2_3_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_2_4_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_2_5_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_2_6_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_2_7_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_2_8_lr0.005_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_2_9_lr0.001_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_3_0_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_3_1_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_3_2_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_3_3_lr0.001_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_3_4_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GAT_SIR_3_5_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_3_6_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_3_7_lr0.001_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_3_8_lr0.001_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_3_9_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_4_0_lr0.001_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GAT_SIR_4_1_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_4_2_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_4_3_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_4_4_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_4_5_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GAT_SIR_4_6_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GAT_SIR_4_7_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GAT_SIR_4_8_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GAT_SIR_4_9_lr0.001_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_0_0_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_0_1_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_0_2_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_0_3_lr0.001_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_0_4_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_0_5_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_0_6_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_0_7_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_0_8_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_0_9_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_1_0_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_1_1_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_1_2_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_1_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_1_4_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_1_5_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_1_6_lr0.005_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_1_7_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_1_8_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_1_9_lr0.005_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_2_0_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_2_1_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_2_2_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_2_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_2_4_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_2_5_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_2_6_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_2_7_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_2_8_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_2_9_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_3_0_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_3_1_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_3_2_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_3_3_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_3_4_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_3_5_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_3_6_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_3_7_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_3_8_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_3_9_lr0.001_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_4_0_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_4_1_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_4_2_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_4_3_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_IC_4_4_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_4_5_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_4_6_lr0.001_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_4_7_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_IC_4_8_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_IC_4_9_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_0_0_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_0_1_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_2_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_3_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_4_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_5_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_6_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_7_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_0_8_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_0_9_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_1_0_lr0.01_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_1_1_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_1_2_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_1_3_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_1_4_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_1_5_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_1_6_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_1_7_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_1_8_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_1_9_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_0_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_1_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_2_2_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_4_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_2_5_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_2_6_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_7_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_8_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_2_9_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_3_0_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_1_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_2_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_3_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_4_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_3_5_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_3_6_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SI_3_7_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_8_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_3_9_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_0_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_4_1_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_2_lr0.001_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_3_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_4_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SI_4_5_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_6_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_7_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_8_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SI_4_9_lr0.01_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_0_0_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_0_1_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_0_2_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_0_3_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_0_4_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_0_5_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_0_6_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_0_7_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_0_8_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_0_9_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_1_0_lr0.01_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_1_1_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_1_2_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_1_3_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_1_4_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_1_5_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_1_6_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_1_7_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_1_8_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_1_9_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_2_0_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_2_1_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_2_2_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_2_3_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_2_4_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_2_5_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_2_6_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_2_7_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_2_8_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_2_9_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_3_0_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_1_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_2_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_3_3_lr0.005_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_3_4_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_5_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_3_6_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_7_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_8_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_GraphSAGE_SIR_3_9_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_0_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_1_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_4_2_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_4_3_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_4_4_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_5_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_6_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_7_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_GraphSAGE_SIR_4_8_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_GraphSAGE_SIR_4_9_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_IC_0_0_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_IC_0_1_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_IC_0_2_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_IC_0_3_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_IC_0_4_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_IC_0_5_lr0.005_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_IC_0_6_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_IC_0_7_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_IC_0_8_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_IC_0_9_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_IC_1_0_lr0.01_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_IC_1_1_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_IC_1_2_lr0.001_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_IC_1_3_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_IC_1_4_lr0.01_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_IC_1_5_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_IC_1_6_lr0.001_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_IC_1_7_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_IC_1_8_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_IC_1_9_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_IC_2_0_lr0.01_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_IC_2_1_lr0.005_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_IC_2_2_lr0.005_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_IC_2_3_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_IC_2_4_lr0.001_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_IC_2_5_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_IC_2_6_lr0.001_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_IC_2_7_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_IC_2_8_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_IC_2_9_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_IC_3_0_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_IC_3_1_lr0.01_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_IC_3_2_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_IC_3_3_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_IC_3_4_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_IC_3_5_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_IC_3_6_lr0.001_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_IC_3_7_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_IC_3_8_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_IC_3_9_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_IC_4_0_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_IC_4_1_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_IC_4_2_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_IC_4_3_lr0.005_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_IC_4_4_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_IC_4_5_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_IC_4_6_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_IC_4_7_lr0.001_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_IC_4_8_lr0.005_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_IC_4_9_lr0.001_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SI_0_0_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_0_1_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_0_2_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SI_0_3_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_0_4_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_0_5_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SI_0_6_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_0_7_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_SI_0_8_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_SI_0_9_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_SI_1_0_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_1_1_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SI_1_2_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_1_3_lr0.005_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_SI_1_4_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_1_5_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_SI_1_6_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_1_7_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SI_1_8_lr0.005_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_SI_1_9_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SI_2_0_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_1_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_2_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_3_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_4_lr0.001_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_5_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_SI_2_6_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_2_7_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SI_2_8_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_2_9_lr0.005_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_0_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_1_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SI_3_2_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_3_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_4_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_SI_3_5_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SI_3_6_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_7_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_3_8_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_SI_3_9_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_0_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_1_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SI_4_2_lr0.001_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_3_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SI_4_4_lr0.005_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SI_4_5_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_6_lr0.01_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_SI_4_7_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_8_lr0.01_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SI_4_9_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SIR_0_0_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_SIR_0_1_lr0.01_wd0.001_hd64_dp0.6.pt",
    "Nuevo_model_SIR_0_2_lr0.01_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SIR_0_3_lr0.01_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_SIR_0_4_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SIR_0_5_lr0.005_wd0.01_hd128_dp0.6.pt",
    "Nuevo_model_SIR_0_6_lr0.01_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SIR_0_7_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_SIR_0_8_lr0.01_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_SIR_0_9_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_1_0_lr0.001_wd0.001_hd64_dp0.4.pt",
    "Nuevo_model_SIR_1_1_lr0.01_wd0.001_hd64_dp0.2.pt",
    "Nuevo_model_SIR_1_2_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_1_3_lr0.005_wd0.01_hd64_dp0.6.pt",
    "Nuevo_model_SIR_1_4_lr0.001_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_SIR_1_5_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SIR_1_6_lr0.001_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_SIR_1_7_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SIR_1_8_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_SIR_1_9_lr0.005_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_SIR_2_0_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SIR_2_1_lr0.01_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_SIR_2_2_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_2_3_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_2_4_lr0.01_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_SIR_2_5_lr0.01_wd0.1_hd64_dp0.4.pt",
    "Nuevo_model_SIR_2_6_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SIR_2_7_lr0.005_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SIR_2_8_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SIR_2_9_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_SIR_3_0_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SIR_3_1_lr0.01_wd0.01_hd64_dp0.4.pt",
    "Nuevo_model_SIR_3_2_lr0.005_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SIR_3_3_lr0.01_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SIR_3_4_lr0.01_wd0.1_hd128_dp0.6.pt",
    "Nuevo_model_SIR_3_5_lr0.001_wd0.01_hd128_dp0.2.pt",
    "Nuevo_model_SIR_3_6_lr0.005_wd0.001_hd128_dp0.6.pt",
    "Nuevo_model_SIR_3_7_lr0.005_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_SIR_3_8_lr0.001_wd0.001_hd128_dp0.4.pt",
    "Nuevo_model_SIR_3_9_lr0.01_wd0.1_hd64_dp0.2.pt",
    "Nuevo_model_SIR_4_0_lr0.005_wd0.01_hd128_dp0.4.pt",
    "Nuevo_model_SIR_4_1_lr0.01_wd0.001_hd128_dp0.2.pt",
    "Nuevo_model_SIR_4_2_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SIR_4_3_lr0.001_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_4_4_lr0.005_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_SIR_4_5_lr0.005_wd0.1_hd128_dp0.2.pt",
    "Nuevo_model_SIR_4_6_lr0.005_wd0.1_hd128_dp0.4.pt",
    "Nuevo_model_SIR_4_7_lr0.005_wd0.01_hd64_dp0.2.pt",
    "Nuevo_model_SIR_4_8_lr0.01_wd0.1_hd64_dp0.6.pt",
    "Nuevo_model_SIR_4_9_lr0.01_wd0.001_hd128_dp0.2.pt"
  ]






# Función para asignar nombres de archivos a las listas correspondientes
def asignar_modelo_a_lista(filename):
    if "GAT" in filename:
        if "SIR" in filename:
            modelos_paths_GAT_SIR.append(filename)
        elif "SI" in filename:
            modelos_paths_GAT_SI.append(filename)
        elif "IC" in filename:
            modelos_paths_GAT_IC.append(filename)
    elif "GraphSAGE" in filename:
        if "SIR" in filename:
            modelos_paths_GraphSAGE_SIR.append(filename)
        elif "SI" in filename:
            modelos_paths_GraphSAGE_SI.append(filename)
        elif "IC" in filename:
            modelos_paths_GraphSAGE_IC.append(filename)
    elif "SIR" in filename and "GAT" not in filename and "GraphSAGE" not in filename:
        modelos_paths_GCN_SIR.append(filename)
    elif "SI" in filename and "GAT" not in filename and "GraphSAGE" not in filename:
        modelos_paths_GCN_SI.append(filename)
    elif "IC" in filename and "GAT" not in filename and "GraphSAGE" not in filename:
        modelos_paths_GCN_IC.append(filename)

# Asignar cada nombre de archivo a la lista correspondiente
for model_name in model_names:
    asignar_modelo_a_lista(model_name)

# Mostrar los resultados en una tabla
modelos_dict = {
    "GCN_SIR": modelos_paths_GCN_SIR,
    "GCN_SI": modelos_paths_GCN_SI,
    "GCN_IC": modelos_paths_GCN_IC,
    "GAT_SIR": modelos_paths_GAT_SIR,
    "GAT_SI": modelos_paths_GAT_SI,
    "GAT_IC": modelos_paths_GAT_IC,
    "GraphSAGE_SIR": modelos_paths_GraphSAGE_SIR,
    "GraphSAGE_SI": modelos_paths_GraphSAGE_SI,
    "GraphSAGE_IC": modelos_paths_GraphSAGE_IC
}

import pandas as pd
modelos_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in modelos_dict.items()]))

# Mostrar el DataFrame
print(modelos_df)

Las siguientes funciones sirven para cargar los modelos resultantes del entrenamiento, para los modelos GCN entrenados con los distintos modelos de difusión.

In [20]:
import re
import torch

def cargar_modelo_GCN_SIR(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_SIR_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo para SIR")

    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GCN_IC(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_IC_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo para IC")

    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GCN_SI(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_SI_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo para SI")

    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model


Las siguientes celdas realizan la selección de la mejor red de entrenamiento usando para ello las redes de validación según la ecuación mostrada en la memoria.

In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SIR = [cargar_modelo_GCN_SIR(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GCN_SIR]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SIR = np.zeros((len(modelos_entrenados_SIR), len(val_data_SIR)))

for i, model in enumerate(modelos_entrenados_SIR):
    for j, data_test in enumerate(val_data_SIR):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SIR[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SIR = resultados_influencia_SIR.mean(axis=1)
mejor_red_idx_GCN_SIR = np.argmax(promedio_influencia_por_red_SIR)
print(f"La mejor red de entrenamiento SIR es la número {mejor_red_idx_GCN_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SIR[mejor_red_idx_GCN_SIR]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SI = [cargar_modelo_GCN_SI(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GCN_SI]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SI = np.zeros((len(modelos_entrenados_SI), len(val_data_SI)))

for i, model in enumerate(modelos_entrenados_SI):
    for j, data_test in enumerate(val_data_SI):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas, r=0)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SI[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SI = resultados_influencia_SI.mean(axis=1)
mejor_red_idx_GCN_SI = np.argmax(promedio_influencia_por_red_SI)
print(f"La mejor red de entrenamiento SI es la número {mejor_red_idx_GCN_SI + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SI[mejor_red_idx_GCN_SI]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_IC = [cargar_modelo_GCN_IC(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GCN_IC]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_IC = np.zeros((len(modelos_entrenados_IC), len(val_data_IC)))

for i, model in enumerate(modelos_entrenados_IC):
    for j, data_test in enumerate(val_data_IC):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_IC[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_IC = resultados_influencia_IC.mean(axis=1)
mejor_red_idx_GCN_IC = np.argmax(promedio_influencia_por_red_IC)
print(f"La mejor red de entrenamiento IC es la número {mejor_red_idx_GCN_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_IC[mejor_red_idx_GCN_IC]}")


La siguiente es una celda auxiliar usada una vez realizada la selección de la mejor red de entrenamiento en los distintos modelos de difusión. Por ello se deja comentada.

mejor_red_idx_GCN_SIR = 8
mejor_red_idx_GCN_SI = 30
mejor_red_idx_GCN_IC = 40

Ahora se vuelve a hacer de nuevo lo mismo que se ha hecho con GCN para los modelos GAT y GrapSAGE (entrenamiento, funciones para cargar los modelos entrenados y selección de la mejor red de entrenamiento)

In [22]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch.nn import Dropout, CrossEntropyLoss

class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=8, dropout_rate=0.4):
        super(GAT, self).__init__()
        self.dropout_rate = dropout_rate
        self.dropout = Dropout(self.dropout_rate)
        self.gat1 = GATv2Conv(input_dim, hidden_dim, heads=heads, concat=True)
        self.gat2 = GATv2Conv(hidden_dim * heads, output_dim, heads=1, concat=False)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.dropout(x)
        x = F.elu(self.gat1(x, edge_index))
        x = self.dropout(x)
        x = self.gat2(x, edge_index)
        return x


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GAT_SIR = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SIR):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GAT(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GAT_SIR_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GAT_SIR.append(model_path)
        print(f"Modelo SIR {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GAT_SI = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SI):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GAT(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GAT_SI_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GAT_SI.append(model_path)
        print(f"Modelo SI {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GAT_IC = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_IC):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GAT(input_dim=data.num_features, hidden_dim=hidden_dim, output_dim=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GAT_IC_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GAT_IC.append(model_path)
        print(f"Modelo IC {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [24]:
import re
import torch

def cargar_modelo_GAT_SIR(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GAT_SIR_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo extraer los hiperparámetros del nombre del archivo para SIR")
    
    model = GAT(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, heads=8, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GAT_SI(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GAT_SI_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo extraer los hiperparámetros del nombre del archivo para SI")
    
    model = GAT(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, heads=8, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GAT_IC(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GAT_IC_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo para IC")
    
    model = GAT(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, heads=8, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SIR = [cargar_modelo_GAT_SIR(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GAT_SIR]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SIR = np.zeros((len(modelos_entrenados_SIR), len(val_data_SIR)))

for i, model in enumerate(modelos_entrenados_SIR):
    for j, data_test in enumerate(val_data_SIR):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SIR[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SIR = resultados_influencia_SIR.mean(axis=1)
mejor_red_idx_GAT_SIR = np.argmax(promedio_influencia_por_red_SIR)
print(f"La mejor red de entrenamiento SIR es la número {mejor_red_idx_GAT_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SIR[mejor_red_idx_GAT_SIR]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SI = [cargar_modelo_GAT_SI(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GAT_SI]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SI = np.zeros((len(modelos_entrenados_SI), len(val_data_SI)))

for i, model in enumerate(modelos_entrenados_SI):
    for j, data_test in enumerate(val_data_SI):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas, r=0)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SI[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SI = resultados_influencia_SI.mean(axis=1)
mejor_red_idx_GAT_SI = np.argmax(promedio_influencia_por_red_SI)
print(f"La mejor red de entrenamiento SI es la número {mejor_red_idx_GAT_SI + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SI[mejor_red_idx_GAT_SI]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_IC = [cargar_modelo_GAT_IC(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GAT_IC]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_IC = np.zeros((len(modelos_entrenados_IC), len(val_data_IC)))

for i, model in enumerate(modelos_entrenados_IC):
    for j, data_test in enumerate(val_data_IC):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_IC[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_IC = resultados_influencia_IC.mean(axis=1)
mejor_red_idx_GAT_IC = np.argmax(promedio_influencia_por_red_IC)
print(f"La mejor red de entrenamiento IC es la número {mejor_red_idx_GAT_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_IC[mejor_red_idx_GAT_IC]}")


mejor_red_idx_GAT_SIR = 20
mejor_red_idx_GAT_SI = 48
mejor_red_idx_GAT_IC = 24

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout_rate=0.5):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        # Capa de entrada
        self.convs.append(SAGEConv(input_dim, hidden_dim))
        
        # Capas intermedias
        for i in range(1, num_layers-1):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        
        # Capa de salida
        self.convs.append(SAGEConv(hidden_dim, output_dim))
        self.dropout = nn.Dropout(p=dropout_rate)  # Configurable dropout probability
    
    def forward(self, x, edge_index):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        x = self.convs[-1](x, edge_index)
        return x


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GraphSAGE_SIR = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SIR):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GraphSAGE(in_features=data.num_features, hidden_dim=hidden_dim, out_features=2, dropout_rate=dropout_rate).to(device)
        
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data.x, data.edge_index)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GraphSAGE_SIR_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GraphSAGE_SIR.append(model_path)
        print(f"Modelo SIR {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GraphSAGE_SI = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_SI):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GraphSAGE(in_features=data.num_features, hidden_dim=hidden_dim, out_features=2, dropout_rate=dropout_rate).to(device)
        
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data.x, data.edge_index)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GraphSAGE_SI_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GraphSAGE_SI.append(model_path)
        print(f"Modelo SI {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm
from torch.nn import Dropout, CrossEntropyLoss
import itertools
import random

modelos_paths_GraphSAGE_IC = []

# Define los rangos de hiperparámetros
lrs = [0.01, 0.005, 0.001]
weight_decays = [0.1, 0.01, 0.001]
hidden_dims = [64, 128]
dropout_rates = [0.2, 0.4, 0.6]  # Agregar más valores según se considere necesario
patience = 50  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora


# Bucle sobre cada red en el conjunto de entrenamiento
for idx, data in enumerate(train_data_IC):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)
    num_nodes = data.num_nodes

    # Variables para guardar el mejor modelo y error de cada configuración
    best_global_val_loss = float('inf')
    best_global_params = {}

    # Prueba todas las combinaciones de hiperparámetros
    for lr, weight_decay, hidden_dim, dropout_rate in itertools.product(lrs, weight_decays, hidden_dims, dropout_rates):
        print(f"Testing combination: lr={lr}, wd={weight_decay}, hd={hidden_dim}, dp={dropout_rate}")
        model = GraphSAGE(in_features=data.num_features, hidden_dim=hidden_dim, out_features=2, dropout_rate=dropout_rate).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = CrossEntropyLoss()

        # Crear máscaras de entrenamiento y validación
        train_mask = torch.rand(num_nodes) < 0.8
        val_mask = ~train_mask
        data.train_mask = train_mask
        data.val_mask = val_mask
        
        patience_counter = 0
        best_val_loss = float('inf')
        best_model = None

        for epoch in range(1000):  # Considera usar un número menor de épocas si el entrenamiento es muy largo
            model.train()
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                out = model(data.x, data.edge_index)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Guardar el mejor modelo para esta combinación de parámetros
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch} with best val_loss {best_val_loss}")
                break

        # Comparar contra el mejor error de validación global
        if best_val_loss < best_global_val_loss:
            best_global_val_loss = best_val_loss
            best_global_params = {'lr': lr, 'weight_decay': weight_decay, 'hidden_dim': hidden_dim, 'dropout': dropout_rate, 'idx': idx}
            best_global_model = best_model

    if best_global_model is not None:
        model_path = f"Nuevo_model_GraphSAGE_IC_{chunk_index}_{idx}_lr{best_global_params['lr']}_wd{best_global_params['weight_decay']}_hd{best_global_params['hidden_dim']}_dp{best_global_params['dropout']}.pt"
        torch.save(best_global_model.state_dict(), model_path)
        modelos_paths_GraphSAGE_IC.append(model_path)
        print(f"Modelo IC {idx+1} guardado en {model_path} con lr={best_global_params['lr']}, weight_decay={best_global_params['weight_decay']}, hidden_dim={best_global_params['hidden_dim']}, dropout={best_global_params['dropout']}")



In [27]:
import re
import torch

def cargar_modelo_GraphSAGE_SIR(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GraphSAGE_SIR_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo extraer los hiperparámetros del nombre del archivo para SIR")
    
    model = GraphSAGE(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GraphSAGE_SI(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GraphSAGE_SI_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo extraer los hiperparámetros del nombre del archivo para SI")
    
    model = GraphSAGE(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GraphSAGE_IC(path, input_dim, output_dim, device):
    match = re.search(r'Nuevo_model_GraphSAGE_IC_\d+_\d+_lr([\d\.]+)_wd([\d\.]+)_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        lr = float(match.group(1))
        weight_decay = float(match.group(2))
        hidden_dim = int(match.group(3))
        dropout_rate = float(match.group(4))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo para IC")
    
    model = GraphSAGE(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SIR = [cargar_modelo_GraphSAGE_SIR(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GraphSAGE_SIR]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SIR = np.zeros((len(modelos_entrenados_SIR), len(val_data_SIR)))

for i, model in enumerate(modelos_entrenados_SIR):
    for j, data_test in enumerate(val_data_SIR):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        # Asegúrate de que los datos estén en el mismo dispositivo que el modelo
        data_test = Data(x=x_test, edge_index=edge_index).to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)  # Asegúrate de usar 'data_test'
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SIR[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SIR = resultados_influencia_SIR.mean(axis=1)
mejor_red_idx_GraphSAGE_SIR = np.argmax(promedio_influencia_por_red_SIR)
print(f"La mejor red de entrenamiento SIR es la número {mejor_red_idx_GraphSAGE_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SIR[mejor_red_idx_GraphSAGE_SIR]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_SI = [cargar_modelo_GraphSAGE_SI(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GraphSAGE_SI]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_SI = np.zeros((len(modelos_entrenados_SI), len(val_data_SI)))

for i, model in enumerate(modelos_entrenados_SI):
    for j, data_test in enumerate(val_data_SI):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index).to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)  # Usar data_test aquí
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas, r=0)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SI[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SI = resultados_influencia_SI.mean(axis=1)
mejor_red_idx_GraphSAGE_SI = np.argmax(promedio_influencia_por_red_SI)
print(f"La mejor red de entrenamiento SI es la número {mejor_red_idx_GraphSAGE_SI + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SI[mejor_red_idx_GraphSAGE_SI]}")


In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """Verifica si el objeto es un string que representa un dígito."""
    return isinstance(s, str) and s.isdigit()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelos_entrenados_IC = [cargar_modelo_GraphSAGE_IC(path, input_dim=64, output_dim=2, device=device) for path in modelos_paths_GraphSAGE_IC]

# Asumir que 'val_data' es una lista de objetos Data utilizados para la validación
resultados_influencia_IC = np.zeros((len(modelos_entrenados_IC), len(val_data_IC)))

for i, model in enumerate(modelos_entrenados_IC):
    for j, data_test in enumerate(val_data_IC):
        idx = val_indices[j]
        G_test = to_networkx(data_test, to_undirected=True)

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'val_data/embeddings_{idx}.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index).to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)  # Usar data_test aquí
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_IC[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_IC = resultados_influencia_IC.mean(axis=1)
mejor_red_idx_GraphSAGE_IC = np.argmax(promedio_influencia_por_red_IC)
print(f"La mejor red de entrenamiento IC es la número {mejor_red_idx_GraphSAGE_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_IC[mejor_red_idx_GraphSAGE_IC]}")



mejor_red_idx_GraphSAGE_SIR = 17
mejor_red_idx_GraphSAGE_SI = 34
mejor_red_idx_GraphSAGE_IC = 29

La siguiente celda crea las funciones para cargar los modelos de las mejores redes de entrenamiento según el índice que ocupa en la lista de redes de entrenamiento de cada modelo de difusión distinto. Cada funcion sirve para cargar los modelos según la variante de GNN utilizada (GCN, GAT o GraphSAGE).

In [29]:
import re
import torch

def cargar_modelo_GCN_generic(paths, idx, input_dim, output_dim, device):
    path = paths[idx]
    match = re.search(r'Nuevo_model_(\w+)_\d+_\d+_lr[\d\.]+_wd[\d\.]+_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        hidden_dim = int(match.group(2))
        dropout_rate = float(match.group(3))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo")
    
    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GAT_generic(paths, idx, input_dim, output_dim, device):
    path = paths[idx]
    match = re.search(r'Nuevo_model_GAT_(\w+)_\d+_\d+_lr[\d\.]+_wd[\d\.]+_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        hidden_dim = int(match.group(2))
        dropout_rate = float(match.group(3))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo")
    
    model = GAT(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, heads=8, dropout_rate=dropout_rate)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model

def cargar_modelo_GraphSAGE_generic(paths, idx, input_dim, output_dim, device):
    path = paths[idx]
    match = re.search(r'Nuevo_model_GraphSAGE_(\w+)_\d+_\d+_lr[\d\.]+_wd[\d\.]+_hd(\d+)_dp([\d\.]+).pt', path)
    if match:
        hidden_dim = int(match.group(2))
        dropout_rate = float(match.group(3))
    else:
        raise ValueError("No se pudo interpretar los hiperparámetros del nombre del archivo")
    
    model = GraphSAGE(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=3, dropout_rate=dropout_rate)  # Assuming fixed 3 layers
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model



La siguiente celda crea las gráficas que muestran la influencia promedio de cada red de test según los distintos tamaños del conjunto de semillas iniciales ejecutados (10,15,....,50).

In [30]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import networkx as nx
from torch_geometric.data import Data
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def seleccionar_semillas(modelo, data, size, model_type):
    modelo.eval()
    with torch.no_grad():
        if model_type == 'GCN' or model_type == 'GAT':
            out = modelo(data)  # Para GNN que espera un objeto Data
        else:
            out = modelo(data.x, data.edge_index)  # Para GAT y GraphSAGE que esperan x y edge_index por separado
        influencia_scores = torch.softmax(out, dim=1)[:, 1]
        _, indices_semillas = torch.topk(influencia_scores, size)
        semillas = indices_semillas.tolist()
    return semillas

# Modelos de GNN
modelos_gnn_sir = {
    'GCN': (cargar_modelo_GCN_generic(modelos_paths_GCN_SIR, mejor_red_idx_GCN_SIR, 64, 2, device), 'GCN'),
    'GAT': (cargar_modelo_GAT_generic(modelos_paths_GAT_SIR, mejor_red_idx_GAT_SIR, 64, 2, device), 'GAT'),
    'GraphSAGE': (cargar_modelo_GraphSAGE_generic(modelos_paths_GraphSAGE_SIR, mejor_red_idx_GraphSAGE_SIR, 64, 2, device), 'GraphSAGE')
}

modelos_gnn_ic = {
    'GCN': (cargar_modelo_GCN_generic(modelos_paths_GCN_IC, mejor_red_idx_GCN_IC, 64, 2, device), 'GCN'),
    'GAT': (cargar_modelo_GAT_generic(modelos_paths_GAT_IC, mejor_red_idx_GAT_IC, 64, 2, device), 'GAT'),
    'GraphSAGE': (cargar_modelo_GraphSAGE_generic(modelos_paths_GraphSAGE_IC, mejor_red_idx_GraphSAGE_IC, 64, 2, device), 'GraphSAGE')
}

modelos_gnn_si = {
    'GCN': (cargar_modelo_GCN_generic(modelos_paths_GCN_SI, mejor_red_idx_GCN_SI, 64, 2, device), 'GCN'),
    'GAT': (cargar_modelo_GAT_generic(modelos_paths_GAT_SI, mejor_red_idx_GAT_SI, 64, 2, device), 'GAT'),
    'GraphSAGE': (cargar_modelo_GraphSAGE_generic(modelos_paths_GraphSAGE_SI, mejor_red_idx_GraphSAGE_SI, 64, 2, device), 'GraphSAGE')
}


# Función para seleccionar semillas usando modelos clásicos
def seleccionar_semillas_clasico(G, size, model_type):
    if model_type == 'degree':
        # Seleccionar nodos con mayor grado
        degree_centrality = nx.degree_centrality(G)
        semillas = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:size]
    elif model_type == 'betweenness':
        # Seleccionar nodos con mayor centralidad de intermediación
        betweenness_centrality = nx.betweenness_centrality(G)
        semillas = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:size]
    return semillas

# Añadir modelos clásicos a la lista de modelos
modelos_clasicos = {
    'Degree': 'degree',
    'Betweenness': 'betweenness'
}

sizes = range(10, 51, 5)

# Preparar DataFrame para guardar los resultados
columnas = [f'Size_{size}' for size in sizes]
indice_modelos_SIR = ['GCN_SIR', 'GAT_SIR', 'GraphSAGE_SIR', 'Degree', 'Betweenness']
df_resultados_SIR = pd.DataFrame(columns=columnas, index=indice_modelos_SIR)
df_resultados_SIR_sinteticas = pd.DataFrame(columns=columnas, index=indice_modelos_SIR)

indice_modelos_SI = ['GCN_SI', 'GAT_SI', 'GraphSAGE_SI', 'Degree', 'Betweenness']
df_resultados_SI = pd.DataFrame(columns=columnas, index=indice_modelos_SI)
df_resultados_SI_sinteticas = pd.DataFrame(columns=columnas, index=indice_modelos_SI)

indice_modelos_IC = ['GCN_IC', 'GAT_IC', 'GraphSAGE_IC', 'Degree', 'Betweenness']
df_resultados_IC = pd.DataFrame(columns=columnas, index=indice_modelos_IC)
df_resultados_IC_sinteticas = pd.DataFrame(columns=columnas, index=indice_modelos_IC)



def plot_influencia(resultados, sizes, title, index, diffusion_type):
    plt.figure(figsize=(10, 6))
    for key, values in resultados.items():
        plt.plot(sizes, values, label=key, marker='o')
    plt.title(title, fontsize=16)
    plt.xlabel('Número de Semillas Iniciales', fontsize=14)
    plt.ylabel('Influencia Promedio Normalizada', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.savefig(f'./resultados_TFG/Grafica_Influencia_{diffusion_type}_todos_vs_inicial_Red_{index}.png')
    plt.close()

def plot_influencia_01(resultados, sizes, title, index, diffusion_type):
    plt.figure(figsize=(10, 6))
    for key, values in resultados.items():
        plt.plot(sizes, values, label=key, marker='o')
    plt.title(title, fontsize=16)
    plt.xlabel('Número de Semillas Iniciales', fontsize=16)
    plt.ylabel('Influencia Promedio Normalizada', fontsize=16)
    plt.ylim([0, 1.1])
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.savefig(f'./resultados_TFG/Grafica_Influencia_{diffusion_type}_todos_vs_inicial_Red_{index}_01.png')
    plt.close()
    
    
    
    
nombres_redes = [
    "Wiki",
    "Fb-Food",
    "Web-edu",
    "Fb-Messages",
    "USAir 500",
    "Hamster",
    "Bitcoin",
    "Mahindas"
]

    
    
# #Inicializar DataFrame para acumular resultados
df_acumulado_SIR_sinteticas = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])    

# Proceso para cada red Barabási-Albert indexada de 0 a 5
for index in range(6):  # Asumiendo que index recorre las redes Barabási-Albert
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_sir, **modelos_clasicos}}
    G_test = redes_test[index]  # Suponemos que esto ya es un grafo de networkx

    # Convertir G_test de networkx a Data de PyTorch Geometric si necesario
    if not isinstance(G_test, Data):
        G_test = from_networkx(G_test)
    
    G_test = to_networkx(G_test, to_undirected=True)  # Así nos aseguramos de que es manipulable como un grafo de networkx
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    num_nodes = len(G_test.nodes)
    edge_index = torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous()
    
    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_sir[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_SIR(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_SIR_sinteticas.empty:
        df_acumulado_SIR_sinteticas = df_resultados_temp
    else:
        df_acumulado_SIR_sinteticas += df_resultados_temp

    
    
    # Generar gráfica para cada red
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en SIR vs. Tamaño de Semillas Iniciales para red sintética', index, 'SIR')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_SIR_sinteticas = df_acumulado_SIR_sinteticas / 8

# Guardar los resultados promedio
df_resultados_promedio_SIR_sinteticas.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_SIR_sinteticas.csv')




#Inicializar DataFrame para acumular resultados
df_acumulado_SIR = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])
    
for index in range(6, 14):
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_sir, **modelos_clasicos}}
    G_test = redes_test[index]
    G_test = to_networkx(G_test, to_undirected=True)
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_sir[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_SIR(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_SIR.empty:
        df_acumulado_SIR = df_resultados_temp
    else:
        df_acumulado_SIR += df_resultados_temp

    nombre_red = nombres_redes[index - 6]
    
    # Generar gráfica para cada red
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en SIR vs. Tamaño de Semillas Iniciales para {nombre_red}', index, 'SIR')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_SIR = df_acumulado_SIR / 8

# Guardar los resultados promedio
df_resultados_promedio_SIR.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_SIR.csv')




#Inicializar DataFrame para acumular resultados
df_acumulado_SI_sinteticas = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])

# Proceso para cada red Barabási-Albert indexada de 0 a 5
for index in range(6):  # Asumiendo que index recorre las redes Barabási-Albert
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_si, **modelos_clasicos}}
    G_test = redes_test[index]  # Suponemos que esto ya es un grafo de networkx

    # Convertir G_test de networkx a Data de PyTorch Geometric si necesario
    if not isinstance(G_test, Data):
        G_test = from_networkx(G_test)
    
    G_test = to_networkx(G_test, to_undirected=True)  # Así nos aseguramos de que es manipulable como un grafo de networkx
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    num_nodes = len(G_test.nodes)
    edge_index = torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous()
    
    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_si[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_SIR(G_test, semillas, r=0)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_SI_sinteticas.empty:
        df_acumulado_SI_sinteticas = df_resultados_temp
    else:
        df_acumulado_SI_sinteticas += df_resultados_temp

    # Generar gráfica para cada red
    plot_influencia(resultados, list(sizes), f'Influencia Promedio en SI vs. Tamaño de Semillas Iniciales para Red {index}', index, 'SI')
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en SI vs. Tamaño de Semillas Iniciales para Red {index}', index, 'SI')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_SI_sinteticas = df_acumulado_SI_sinteticas / 6

# Guardar los resultados promedio
df_resultados_promedio_SI_sinteticas.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_SI_sinteticas.csv')




    
# Inicializar DataFrame para acumular resultados
df_acumulado_SI = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])

# Bucle sobre las redes
for index in range(6, 14):
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_si, **modelos_clasicos}}
    G_test = redes_test[index]
    G_test = to_networkx(G_test, to_undirected=True)
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_si[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_SIR(G_test, semillas, r=0)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_SI.empty:
        df_acumulado_SI = df_resultados_temp
    else:
        df_acumulado_SI += df_resultados_temp

    # Generar gráfica para cada red
    plot_influencia(resultados, list(sizes), f'Influencia Promedio en SI vs. Tamaño de Semillas Iniciales para Red {index}', index, 'SI')
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en SI vs. Tamaño de Semillas Iniciales para Red {index}', index, 'SI')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_SI = df_acumulado_SI / 8

# Guardar los resultados promedio
df_resultados_promedio_SI.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_SI.csv')
   
    
#Inicializar DataFrame para acumular resultados
df_acumulado_IC_sinteticas = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])    

# Proceso para cada red Barabási-Albert indexada de 0 a 5
for index in range(6):  # Asumiendo que index recorre las redes Barabási-Albert
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_ic, **modelos_clasicos}}
    G_test = redes_test[index]  # Suponemos que esto ya es un grafo de networkx

    # Convertir G_test de networkx a Data de PyTorch Geometric si necesario
    if not isinstance(G_test, Data):
        G_test = from_networkx(G_test)
    
    G_test = to_networkx(G_test, to_undirected=True)  # Así nos aseguramos de que es manipulable como un grafo de networkx
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    num_nodes = len(G_test.nodes)
    edge_index = torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous()
    
    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_ic[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_IC(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_IC_sinteticas.empty:
        df_acumulado_IC_sinteticas = df_resultados_temp
    else:
        df_acumulado_IC_sinteticas += df_resultados_temp

    # Generar gráfica para cada red
    plot_influencia(resultados, list(sizes), f'Influencia Promedio en IC vs. Tamaño de Semillas Iniciales para Red {index}', index, 'IC')
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en IC vs. Tamaño de Semillas Iniciales para Red {index}', index, 'IC')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_IC_sinteticas = df_acumulado_IC_sinteticas / 6

# Guardar los resultados promedio
df_resultados_promedio_IC_sinteticas.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_IC_sinteticas.csv')     
    

    
# Inicializar DataFrame para acumular resultados
df_acumulado_IC = pd.DataFrame(columns=[f'Size_{size}' for size in sizes])     
    
# Proceso similar para IC
for index in range(6, 14):
    resultados = {modelo_name: [] for modelo_name in {**modelos_gnn_ic, **modelos_clasicos}}
    G_test = redes_test[index]
    G_test = to_networkx(G_test, to_undirected=True)
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})

    for modelo_name in resultados.keys():
        for size in sizes:
            if modelo_name in modelos_clasicos:
                semillas = seleccionar_semillas_clasico(G_test, size, modelos_clasicos[modelo_name])
            else:
                modelo, model_type = modelos_gnn_ic[modelo_name]
                data_test = Data(x=torch.tensor(np.random.rand(len(G_test.nodes()), 64), dtype=torch.float).to(device),
                                 edge_index=torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device))
                semillas = seleccionar_semillas(modelo, data_test, size, model_type)

            infectados_total = 0
            num_runs = 10
            for _ in range(num_runs):
                infectados = simular_difusion_IC(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)

    # Convertir los resultados a DataFrame y sumar al acumulado
    df_resultados_temp = pd.DataFrame(resultados, index=[f'Size_{size}' for size in sizes]).T
    if df_acumulado_IC.empty:
        df_acumulado_IC = df_resultados_temp
    else:
        df_acumulado_IC += df_resultados_temp

    # Generar gráfica para cada red
    plot_influencia(resultados, list(sizes), f'Influencia Promedio en IC vs. Tamaño de Semillas Iniciales para Red {index}', index, 'IC')
    plot_influencia_01(resultados, list(sizes), f'Influencia Promedio en IC vs. Tamaño de Semillas Iniciales para Red {index}', index, 'IC')

    
# Calcular el promedio de influencia para cada modelo y tamaño de semilla
df_resultados_promedio_IC = df_acumulado_IC / 8

# Guardar los resultados promedio
df_resultados_promedio_IC.to_csv('./resultados_TFG/resultados_influencia_normalizada_promedio_IC.csv')    